# 1. 라이브러리

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC


from sklearn.metrics import f1_score, classification_report

import warnings
warnings.filterwarnings('ignore')

# 2. 데이터 불러오기

In [2]:
final_train_path = os.listdir('./data/final_labeling/train')
final_test_path = os.listdir('./data/final_labeling/test')

In [3]:
final_train_path[18], final_test_path[18]

('서울_day_train.csv', '서울_day_test.csv')

서울 데이터를 활용하여 자연재난을 얼마나 정확하게 예측하는지 확인해보겠습니다.

# 3. 학습 데이터/테스트 데이터 분할

In [4]:
col = ['평균_기온', '최저_기온', '최고_기온', '일강수량', '최대_풍속', '최대_풍속_풍향_x', '최대_풍속_풍향_y',
       '평균_풍속', '풍정합', '최다_풍향_x', '최다_풍향_y', '평균_이슬점온도', '최소_상대습도', '평균_상대습도',
       '평균_증기압', '평균_현지기압', '최고_해면_기압', '최저_해면기압', '평균_해면기압', '합계_일조_시간',
       '일_최심신적설', '합계_3시간_신적설', '안개_계속_시간', '체감온도']

In [5]:
# train test split
def data_split(final_train_path, final_test_path, weather, label, col):
    df_train = pd.read_csv('./data/final_labeling/train/' + final_train_path)
    df_test = pd.read_csv('./data/final_labeling/test/' + final_test_path)
    
    df_weather = df_train[df_train['계절'] == weather]
    df_weather.reset_index(drop=True, inplace=True)
    y_train = df_weather[label]
    x_train = df_weather.loc[:, col].copy()
    
    x_test = df_test[df_test['계절'] == weather]
    x_test.reset_index(drop=True, inplace=True)
    y_test = x_test[label]
    x_test = x_test.loc[:, col].copy()

    # 수치형 데이터 정규화
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.fit_transform(x_test)

    x_train = pd.DataFrame(x_train)
    x_train.columns = col

    x_test = pd.DataFrame(x_test)
    x_test.columns = col

    return x_train, x_test, y_train, y_test

학습(train) 데이터와 테스트(test) 데이터로 나눕니다.

StandardScaler를 사용하여 정규화를 진행합니다.

# 4. 머신러닝 모델 설계

In [6]:
# voting
def voting_model(x_train, x_test, y_train, y_test):
    # rf
    rf = RandomForestClassifier()
    param_grid = {'n_estimators' : range(0, 200, 20),
                  'max_depth' : range(5, 10),
                  # 'class_weight' : [{0:1, 1:10}, {0:1, 1:20}],
                  'class_weight' : ['balanced'],
                  # 'random_state' : range(1, 10)
                 }
    gridsearch_rf = GridSearchCV(RandomForestClassifier(), param_grid, scoring='f1_macro')
    gridsearch_rf.fit(x_train, y_train)
    rf_best = gridsearch_rf.best_estimator_
    
    # xgb
    xgb = XGBClassifier()
    param_grid = {'n_estimators' : range(0, 200, 20),
                  'learning_rate' : [1e-3, 1e-2, 1e-1, 1.0, 10.0, 100.0],
                  'max_depth' : range(5, 10),
                  'gamma' : [0, 1, 2, 3],
                  'colsample_bytree' : [0.8, 0.9],
                  # 'random_state' : range(1, 10)
                 }
    gridsearch_xgb = GridSearchCV(XGBClassifier(), param_grid, scoring='f1_macro', n_jobs=-1)
    gridsearch_xgb.fit(x_train, y_train)
    xgb_best = gridsearch_xgb.best_estimator_

    # svc
    svm = SVC()
    param_grid = {'C' : [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                  # 'class_weight' : [{0:1, 1:10}, {0:1, 1:20}],
                  'class_weight' : ['balanced'],
                  'max_iter' : [100, 150, 200, 250, 300],
                  'probability' : [True],
                  # 'random_state' : range(1, 10)
                 }
    gridsearch_svc = GridSearchCV(SVC(), param_grid, scoring='f1_macro')
    gridsearch_svc.fit(x_train, y_train)
    svc_best = gridsearch_svc.best_estimator_

    # lr
    lr = LogisticRegression()
    param_grid = {'C' : [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
                  # 'class_weight' : [{0:1, 1:10}, {0:1, 1:20}],
                  'class_weight' : ['balanced'],
                  'max_iter' : [100, 150, 200, 250, 300],
                  # 'random_state' : range(1, 10)
                 }
    gridsearch_lr = GridSearchCV(LogisticRegression(), param_grid, scoring='f1_macro')
    gridsearch_lr.fit(x_train, y_train)
    lr_best = gridsearch_lr.best_estimator_

    voting_reg = VotingClassifier(
        estimators=[
                    ('lr', lr_best),
                    ('rf', rf_best),
                    ('xgb', xgb_best),
                    ('svc', svc_best),
                   ], voting='soft')
    voting_reg.fit(x_train, y_train)
    y_pred = voting_reg.predict(x_test)
    
    return np.array(y_test), y_pred

LogisticRegression, RandomForestClassifier, XGBClassifier, SVC 모델의 결과들을 집계합니다.

각 분류기의 예측을 평균 내어 확률이 가장 높은 클래스로 예측하는 soft voting을 사용합니다.

# 5-1. 폭염 예측

In [7]:
# col = ['평균_기온', '최저_기온', '최고_기온', '최대_풍속_풍향_x', '최대_풍속_풍향_y',
#        '평균_풍속', '최소_상대습도', '평균_상대습도', '합계_일조_시간', '체감온도', '열대야']
x_train, x_test, y_train, y_test = data_split(final_train_path[18], final_test_path[18], '여름철', '여름_폭염', col)
y_true, y_pred = voting_model(x_train, x_test, y_train, y_test)

In [8]:
print(classification_report(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       269
         1.0       1.00      0.68      0.81        37

    accuracy                           0.96       306
   macro avg       0.98      0.84      0.89       306
weighted avg       0.96      0.96      0.96       306

0.892316715542522


In [9]:
len(y_test[y_test==1]), len(y_pred[y_pred==1])

(37, 25)

In [10]:
# 실제 폭염 라벨링
y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 0.

In [11]:
# 폭염 예측값
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 0., 0.

# 5-2. 한파 예측

In [12]:
# col = ['평균_기온', '최저_기온', '최고_기온', '최대_풍속', '최대_풍속_풍향_x', '최대_풍속_풍향_y', 
#        '평균_풍속', '최다_풍향_x', '최다_풍향_y', '평균_이슬점온도', '합계_일조_시간', '일_최심신적설',
#        '합계_3시간_신적설', '체감온도']
x_train, x_test, y_train, y_test = data_split(final_train_path[18], final_test_path[18], '겨울철', '겨울_한파', col)
y_true, y_pred = voting_model(x_train, x_test, y_train, y_test)

In [13]:
print(classification_report(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       413
         1.0       1.00      0.73      0.84        11

    accuracy                           0.99       424
   macro avg       1.00      0.86      0.92       424
weighted avg       0.99      0.99      0.99       424

0.91924322265253


In [14]:
len(y_test[y_test==1]), len(y_pred[y_pred==1])

(11, 8)

In [15]:
# 실제 한파 라벨링
y_true

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [16]:
# 한파 예측값
y_pred

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# 5-3. 호우 예측

In [17]:
x_train, x_test, y_train, y_test = data_split(final_train_path[18], final_test_path[18], '여름철', '호우', col)
y_true, y_pred = voting_model(x_train, x_test, y_train, y_test)

In [18]:
print(classification_report(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99       294
         1.0       0.67      0.67      0.67        12

    accuracy                           0.97       306
   macro avg       0.83      0.83      0.83       306
weighted avg       0.97      0.97      0.97       306

0.8265306122448979


In [19]:
len(y_test[y_test==1]), len(y_pred[y_pred==1])

(12, 12)

In [20]:
# 실제 호우 라벨링
y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [21]:
# 호우 예측값
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

# 5-4. 대설 예측

In [22]:
# col = ['평균_기온', '최저_기온', '최고_기온', '최대_풍속', '최대_풍속_풍향_x', '최대_풍속_풍향_y', 
#        '평균_풍속', '최다_풍향_x', '최다_풍향_y', '평균_이슬점온도', '합계_일조_시간', '일_최심신적설',
#        '합계_3시간_신적설', '체감온도']
x_train, x_test, y_train, y_test = data_split(final_train_path[18], final_test_path[18], '겨울철', '대설', col)
y_true, y_pred = voting_model(x_train, x_test, y_train, y_test)

In [23]:
print(classification_report(y_true, y_pred))
print(f1_score(y_true, y_pred, average='macro'))

              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00       420
         1.0       1.00      0.25      0.40         4

    accuracy                           0.99       424
   macro avg       1.00      0.62      0.70       424
weighted avg       0.99      0.99      0.99       424

0.6982206405693949


In [24]:
len(y_test[y_test==1]), len(y_pred[y_pred==1])

(4, 1)

In [25]:
# 실제 대설 라벨링
y_true

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [26]:
# 대설 예측값
y_pred

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

실제 라벨링과 예측값이 대체로 일치하지만, 대설 예측값을 보면 데이터 불균형 문제를 완벽하게 해결하지 못한 것처럼 보입니다.

대설의 라벨링처럼 0과 1의 개수가 극심한 차이가 있을 때 데이터 불균형 문제를 더 확실하게 해결할 방법이 필요할 것입니다.